In [1]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct")
persist_directory = "../database/anwesha_chroma"
vectorstore = Chroma(
    persist_directory=persist_directory, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

C:\Users\mdsaz\AppData\Local\Temp\ipykernel_14292\2632607041.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [5]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question": question})
retrieved_docs

[Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content="১০। 'অপরিচিতা' গল্পে মামার সাথে অনুপমের বয়সের পার্থক্য কত?  \n(ক) বছর চারেক (খ) বছর ছয়েক (গ) বছর আটেক (ঘ) বছর দশেক  \n\n১১। বন্যার সিপাহশালার কোনটি শীর্ষক করবেন?  \n(ক) অনুপম চরিত্র (খ) অনুপম সংলাপ  \n(গ) অনুপম রূপসালাপ (ঘ) অনুপম ব্যক্তিত্বসম্পন্ন"),
 Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content='১৪। মামা কেন মেয়ের পক্ষে পক্ষে করতেন?  \n(ক) ধনী (খ) গরিব (গ) প্রমীণ (ঘ) শখলে  \n\n১৫। অনুপমের বন্ধুর নাম কি?  \n(ক) সজল (খ) জ্যোতিষ (গ) হরিশ (ঘ) মণীষ  \n\n১৬। মেয়ের চেয়ে মামার বয়সে খুরবচাই ফার কাছ গুরত্ব?  \n(ক) হরিশের (খ) অনুপমের (গ) মামার (ঘ) ঘটকের'),
 Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content='আমার তখন বয়স তিন বছর। মার হাতেই আমি মানুষ। মা পরিবারের ঘরের মেয়ে; তাই, আমার যে মনি এ কথা তিনিও ভুলেন না, আমাকে ভুলিতে দেন না। শৈশবকালে আমি কোলেই মানুষ-বোধ করি, সেইজন্য এ পর্যন্ত আমার পুরাপুরি বয়সই ইহা নাই।\n\n**আজও আমাকে দেখিলে মনে হইবে, আমি 

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
# HyDE document generation
template = """Please write a short story to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)


generate_docs_for_retrieval = (
    prompt_hyde | ChatGroq(
        model="moonshotai/kimi-k2-instruct",
    ) | StrOutputParser()
)

# Run
question = "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?"
generate_docs_for_retrieval.invoke({"question": question})

'ছোটবেলায় অনুপম প্রায়ই শুনত, “তোমার মামা যখন ক্লাস এইটে পড়ত, তুমি তখন হামাগুড়ি দিতে শেখো।”  \n\nএকদিন সে মামাকে জিজ্ঞেস করল, “মামা, আমি আপনার চেয়ে কত বছরের ছোট?”  \nমামা হেসে বলল, “যখন আমি জন্ম নিলাম, আমার বোন—তোমার মা—ছিল তিন বছরের। আর তিনি যখন তোমার জন্ম দিলেন, তখন আমার বয়স ছিল ঠিক সেই তিন বছর আর আট মাস।”  \n\nঅনুপম হিসেব করে দেখল, মামার জন্মের তিন বছর পর তার মা জন্মেছেন, আর তার মা জন্মের আট মাস পর তিনি নিজে জন্মেছেন। অর্থাৎ মামার চেয়ে সে মোট তিন বছর আর আট মাসের ছোট।\n\nতাই অনুপম তার মামার চেয়ে **৩ বছর ৮ মাস** ছোট ছিল।'

In [6]:
# RAG
llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context": retrieved_docs, "question": question})

"প্রশ্ন: অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?\n\nউত্তর: 'অপরিচিতা' গল্পে মামার সাথে অনুপমের বয়সের পার্থক্য **বছর ছয়েক**।"